# Import Dependencies

In [141]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

from matplotlib import pyplot as plt

from pathlib import Path
from tqdm import tqdm

import warnings

warnings.filterwarnings('ignore')

# Load Data

In [142]:
# load 4 dataframes (returns, nb industries, avg firm size, Sum of BE / Sum of ME)
df = pd.read_csv(str(Path().absolute()) + "/data/48_Industry_Portfolios.CSV") 

# split these dataframes
df_list = np.split(df, df[df.isnull().all(1)].index, axis = 0) 

# clean data and convert date column to index
for i in range(len(df_list)):
    df_list[i] = pd.DataFrame(df_list[i])  
    df_list[i] = df_list[i].dropna()  
    df_list[i].loc[:, "Date"] = df_list[i].loc[:, "Date"].astype("int")  
    df_list[i] = df_list[i].set_index("Date")  
    
    # last data frame has yearly data
    if i == (len(df_list) - 1): 
        df_list[i].index = pd.to_datetime(df_list[i].index, format = "%Y")
        df_list[i].index = df_list[i].index + pd.DateOffset(months = 6)
    else:
        df_list[i].index = pd.to_datetime(df_list[i].index, format = "%Y%m")

# create a dataframe of excess returns, nb of industries and avg sizes
df = df_list[0] / 100
mask = (df <= -0.99)
df[mask] = np.nan

nb_industries = df_list[1]
nb_industries[mask] = np.nan

avg_size = df_list[2]
avg_size[mask] = np.nan

be_over_me = df_list[3]
be_over_me[mask] = np.nan

In [143]:
# market cap of each industry over time
mkt_cap = nb_industries * avg_size
print(mkt_cap.shape)

# book value to market value
be_over_me = be_over_me.resample("1MS").ffill()
print(be_over_me.shape)

# momentum with monthly data
momentum = df.rolling(12).mean()
print(momentum.shape)

(1182, 48)
(1177, 48)
(1182, 48)


# Normalize Data

In [144]:
mkt_cap_ = mkt_cap.loc['1927-06-01':'1973-12-01']
mkt_cap_norm = (mkt_cap_ - mkt_cap_.mean()) / mkt_cap_.std()
print(mkt_cap_norm.shape)

be_over_me_ = be_over_me.loc['1927-06-01':'1973-12-01']
be_over_me_norm = (be_over_me_ - be_over_me_.mean()) / be_over_me_.std()
print(be_over_me_norm.shape)

momentum_ = momentum.loc['1927-06-01':'1973-12-01']
momentum_norm = (momentum_ - momentum_.mean()) / momentum_.std()
print(momentum_norm.shape)

df_in = df.loc['1927-06-01':'1973-12-01']
print(df_in.shape)

(559, 48)
(559, 48)
(559, 48)
(559, 48)


In [145]:
def CRRA(wealth, gamma= 5):
    """"Constant Relative Risk Aversion Utility Function"""

    if gamma == 1:
        return np.log(wealth)
    else:
        return ((1 + wealth) ** (1 - gamma)) / (1 - gamma)


In [146]:
characteristics = np.stack([mkt_cap_norm, be_over_me_norm, momentum_norm], axis= -1)
weights = mkt_cap.iloc[-1]/ mkt_cap.iloc[-1].sum()
theta = np.array([-1.451, 3.606, 1.772])

# Optimization

## Objective

In [147]:
def objective(theta:np.ndarray, x:np.ndarray, rets:pd.DataFrame, weights:np.ndarray):
    accrued_wealth = 0.0
    wealth = 0.0
    for t in range(x.shape[0]):
        # finding the number of Nt per row
        rets_valid = ~pd.isna(rets.iloc[t+1, :])
        x_valid = ~np.isnan(x[t, :, :]).any(axis=1)
        Nt = np.sum(rets_valid & x_valid)
        
        for i in range(x.shape[1]):
            if (np.isnan(rets.iloc[t+1,i])) or (np.isnan(x[t,i,:]).any()):
                continue
            wealth += (weights[i] + theta.T @ x[t,i,:] / Nt) * rets.iloc[t+1,i]
        accrued_wealth += CRRA(wealth)
    
    return - accrued_wealth / x.shape[0]

In [148]:
init = np.array([-1.398e+01,  4.366e+01, 2.770e+01]) # local solution found
# init = theta
response = minimize(objective, x0= init, args= (characteristics, df, weights), method= 'SLSQP')
response

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 6.494698443757973e-06
       x: [-1.398e+01  4.366e+01  2.770e+01]
     nit: 1
     jac: [ 5.138e-07 -4.033e-06  5.735e-06]
    nfev: 4
    njev: 1

# Find New Weights

In [149]:
next_w = np.zeros(weights.shape)
for i in range(len(weights)):
    next_w[i] = weights[i] + (response.x @ characteristics[-1,i,:]) / (np.count_nonzero(~np.isnan(characteristics[-1,:,:]))/3)

next_w

array([-0.15392515, -1.63226366, -2.58114127, -2.3781702 , -2.01743452,
       -1.73431925, -2.48397976, -2.06254471, -2.80009338, -1.48297405,
        1.91309059, -2.8094114 , -2.42926598, -0.22485848, -1.61746641,
       -1.46347063, -0.9922424 , -1.85080179,  0.1245534 , -1.54020126,
       -1.33436336, -1.30295921, -0.76151226, -0.1109189 , -0.20995246,
        1.19035235,  0.43982617, -0.34550187, -0.7077866 , -1.19528758,
       -0.97132281, -0.50712861, -1.87492782, -2.09387426, -2.02236567,
       -1.11023043, -1.26536404, -0.7541866 , -1.89966462, -1.4040221 ,
       -2.04357981, -2.08440924, -2.84235124, -2.03303118, -1.6115431 ,
       -2.03087226, -2.68606177, -4.46803079])

In [150]:
# apply equation 16
def long_only_constraint(weights):
    w_pos = np.clip(weights, 0, None) # set all negative to 0
    return w_pos / np.sum(w_pos)

l_w = long_only_constraint(next_w)
l_w

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.52158756, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0339584 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.32453925, 0.11991479, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

# Out-of-Sample Testing

### 0. Load Out-of-Sample Data

In [151]:
mkt_cap_ = mkt_cap.loc['1963-12-01':'2024-01-01']
mkt_cap_norm = (mkt_cap_ - mkt_cap_.mean()) / mkt_cap_.std()
print(mkt_cap_norm.shape)

be_over_me_ = be_over_me.loc['1963-12-01':'2024-01-01']
be_over_me_norm = (be_over_me_ - be_over_me_.mean()) / be_over_me_.std()
print(be_over_me_norm.shape)

momentum_ = momentum.loc['1963-12-01':'2024-01-01']
momentum_norm = (momentum_ - momentum_.mean()) / momentum_.std()
print(momentum_norm.shape)

df_out = df.loc['1963-12-01':'2024-01-01']
print(df_out.shape)


(722, 48)
(722, 48)
(722, 48)
(722, 48)


In [152]:
x_hat = np.stack([mkt_cap_norm, be_over_me_norm, momentum_norm], axis= -1)
w_bar = mkt_cap.loc['1973-12-01']/ mkt_cap.loc['1973-12-01'].sum()
theta = response.x
theta

array([-13.98,  43.66,  27.7 ])

### Rolling Window

In [153]:
window_size = 120 # 10 years x 12 months = 120 timesteps
next_w = np.zeros(w_bar.shape[0])
monthly_rets = []

for t in tqdm(range(window_size, x_hat.shape[0])): 
    x_hat_subset = x_hat[t-window_size:t, :, :]
    df_out_subset = df_out.iloc[t-window_size:t+1]
    w_bar = mkt_cap_.iloc[t]/ mkt_cap_.iloc[t].sum()

    
    # 1. Estimate Theta
    res = minimize(objective, x0= theta, args= (x_hat_subset, df_out_subset, w_bar), method= 'SLSQP')
    theta = res.x
    # 2. Estimate Weights
    for i in range(len(weights)):
        if np.isnan(x_hat_subset[-1,i,:].any()):
            next_w[i] = 0
            continue
        next_w[i] = weights[i] + (theta @ x_hat_subset[-1,i,:]) / (np.count_nonzero(~np.isnan(x_hat_subset[-1,:,:]))/3)

    long_weights = long_only_constraint(next_w)
    # 3. Estimate Next Month Returns
    rets_clean = np.nan_to_num(df_out.iloc[t+1], nan=0)
    # 4. Record Return
    monthly_rets.append(long_weights @ rets_clean)
    print(f'Date: {df_out.index[t]}, Monthly Return: {long_weights @ rets_clean}')


  0%|          | 1/602 [00:02<22:45,  2.27s/it]

Date: 1973-12-01 00:00:00, Monthly Return: 0.03606207912968548


  0%|          | 2/602 [00:04<22:26,  2.24s/it]

Date: 1974-01-01 00:00:00, Monthly Return: 0.02703442197569069


  0%|          | 3/602 [00:06<22:40,  2.27s/it]

...

Date: 1977-07-01 00:00:00, Monthly Return: -0.0056918097792153805


  7%|▋         | 45/602 [15:49<5:19:30, 34.42s/it]

Date: 1977-08-01 00:00:00, Monthly Return: 0.018768884716791154


  7%|▋         | 45/602 [16:18<3:21:53, 21.75s/it]


KeyboardInterrupt: 